In [9]:
#Utils
import numpy as np
import pandas as pd

#NLP
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import  mark_negation
from nltk.stem import PorterStemmer

In [48]:
#Download resource
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("vader_lexicon")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baspe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\baspe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\baspe\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\baspe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\baspe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [70]:
from nltk.stem.wordnet import WordNetLemmatizer

def apply_lemmatize(word_list):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence_list = []
    for words in word_list:
        lemmatized_sentence = []
        for word, tag in pos_tag(words):
            if tag.startswith('NN'):
                pos = 'n'
            elif tag.startswith('VB'):
                pos = 'v'
            else:
                pos = 'a'
            lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
        lemmatized_sentence_list.append(lemmatized_sentence)
    return lemmatized_sentence_list

In [0]:
hospital_list = ["bumrungrad",  "lerdsin", "rajavithi"]

In [71]:
df = pd.read_csv("lerdsin-translated.csv")

In [72]:
df.head()

,reviews,ratings,reviews_en
0,จำไม่ได้ว่าปีไหน น่าจะ ช่วงปลายปี 57 หรือ 58 ต...,1,I can't remember which year it was at the end ...
1,เคยใช้บริการตอนขาหัก โดยใช้สิทธิ์ประกันสังคม ไ...,1,I had a broken leg The right to social securit...
2,รอคิวตรวจนานมากค่ะ มาตั้งแต่ 6 โมงเช้า กว่าจะไ...,1,"Make me wait so long since 6 am to 3 pm, they ..."
3,มาครั้งแรก ประทับใจมากคับ บริการเหมือน รพ เอกช...,1,The first impression is very good services lik...
4,บริการดีครับ แต่คนเยอะมากๆเลย ที่ใช้บริการคือม...,1,"Good service, but a lot of people yet. The ser..."


In [73]:
df["words"] = apply_stopwords(df["reviews_en"])
df.head()

,reviews,ratings,reviews_en,words
0,จำไม่ได้ว่าปีไหน น่าจะ ช่วงปลายปี 57 หรือ 58 ต...,1,I can't remember which year it was at the end ...,"[i, ca, n't, remember, year, end, year, 57, 58..."
1,เคยใช้บริการตอนขาหัก โดยใช้สิทธิ์ประกันสังคม ไ...,1,I had a broken leg The right to social securit...,"[i, broken, leg, the, right, social, security,..."
2,รอคิวตรวจนานมากค่ะ มาตั้งแต่ 6 โมงเช้า กว่าจะไ...,1,"Make me wait so long since 6 am to 3 pm, they ...","[make, wait, long, since, 6, 3, pm, ,, checked..."
3,มาครั้งแรก ประทับใจมากคับ บริการเหมือน รพ เอกช...,1,The first impression is very good services lik...,"[the, first, impression, good, services, like,..."
4,บริการดีครับ แต่คนเยอะมากๆเลย ที่ใช้บริการคือม...,1,"Good service, but a lot of people yet. The ser...","[good, service, ,, lot, people, yet, ., the, s..."


In [74]:
df["lemma_word_of_sentence"] = apply_lemmatize(df["words"])
df.head()

,reviews,ratings,reviews_en,words,lemma_word_of_sentence
0,จำไม่ได้ว่าปีไหน น่าจะ ช่วงปลายปี 57 หรือ 58 ต...,1,I can't remember which year it was at the end ...,"[i, ca, n't, remember, year, end, year, 57, 58...","[i, ca, n't, remember, year, end, year, 57, 58..."
1,เคยใช้บริการตอนขาหัก โดยใช้สิทธิ์ประกันสังคม ไ...,1,I had a broken leg The right to social securit...,"[i, broken, leg, the, right, social, security,...","[i, break, leg, the, right, social, security, ..."
2,รอคิวตรวจนานมากค่ะ มาตั้งแต่ 6 โมงเช้า กว่าจะไ...,1,"Make me wait so long since 6 am to 3 pm, they ...","[make, wait, long, since, 6, 3, pm, ,, checked...","[make, wait, long, since, 6, 3, pm, ,, check, ..."
3,มาครั้งแรก ประทับใจมากคับ บริการเหมือน รพ เอกช...,1,The first impression is very good services lik...,"[the, first, impression, good, services, like,...","[the, first, impression, good, service, like, ..."
4,บริการดีครับ แต่คนเยอะมากๆเลย ที่ใช้บริการคือม...,1,"Good service, but a lot of people yet. The ser...","[good, service, ,, lot, people, yet, ., the, s...","[good, service, ,, lot, people, yet, ., the, s..."
